In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

## init

In [2]:
# this implementation will be specific 
# to a given wind farm layout
# otherwise wouldn't know input size
# b/c input size depends on turbine locations
# akin to how building model only applies to 5-zone building

# here: 2x2 wind turbine layout

class DPCModel(nn.Module):
    def __init__(
        self,
        n_turbines: int,
        hidden_dim: int = 64,
        **kwargs
    ):
        super().__init__()
        input_size = 2 # ws, wd
        self.input_layer = nn.Linear(input_size, hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, n_turbines)
        
    def forward(self, x):
        # x: state of system = torch.tensor([ws, wd])
        # u: control action (yaw angle for each turbine)
        u = F.relu(self.input_layer(x))
        u = F.relu(self.hidden_layer(u))
        u = torch.sigmoid(self.output_layer(u))
        return u

In [3]:
# init test case
umin = torch.tensor([0.]) # min allowed yaw angle (degrees)
umax = torch.tensor([45.]) # max allowed yaw angle (degrees)
model_config = dict({"n_turbines": 4, "hidden_dim": 64})

# init DPC model
model = DPCModel(**model_config)
model

DPCModel(
  (input_layer): Linear(in_features=2, out_features=64, bias=True)
  (hidden_layer): Linear(in_features=64, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=4, bias=True)
)

In [4]:
# hyperparameters -> to do: hyperparameter training
u_penalty = 1.
lr = .1

# init training
opt = torch.optim.Adam(model.parameters(), lr=lr)

## forward pass

In [5]:
# training data
ws = 5.
wd = 10.

# forward pass
x = torch.tensor([ws, wd])
u = model(x)
clipped_u = F.relu(u-umin) + umin
clipped_u = -F.relu(-clipped_u + umax) + umax
clipped_u

tensor([0.4293, 0.4529, 0.7020, 0.3437], grad_fn=<AddBackward0>)

## backward pass : to do

In [6]:
# TO DO: cost from power
# negative sign b/c good -> want to minimize
power_cost = torch.tensor([0.])

In [39]:
ws, wd, model_config["n_turbines"]

(5.0, 10.0, 4)

In [ ]:
# init
thrust = torch.tensor([1.19187945, 1.17284634, 1.09860817, 1.02889592, 0.97373036, 0.92826162,
          0.89210543, 0.86100905, 0.835423, 0.81237673, 0.79225789, 0.77584769,
          0.7629228, 0.76156073, 0.76261984, 0.76169723, 0.75232027, 0.74026851,
          0.72987175, 0.70701647, 0.54054532, 0.45509459, 0.39343381, 0.34250785,
          0.30487242, 0.27164979, 0.24361964, 0.21973831, 0.19918151, 0.18131868,
          0.16537679, 0.15103727, 0.13998636, 0.1289037, 0.11970413, 0.11087113,
          0.10339901, 0.09617888, 0.09009926, 0.08395078, 0.0791188, 0.07448356,
          0.07050731, 0.06684119, 0.06345518, 0.06032267, 0.05741999, 0.05472609])

wind_speed = torch.tensor([2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0,5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5,
              9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 
              15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 
              21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5])

turbine_diameter = 126.0
turbine_radius = turbine_diameter / 2.0
turbine_hub_height = 90.0

x_spc = 5 * 126.0
nturbs = 20
x_coord = np.array([xx * x_spc for xx in range(nturbs)])
y_coord = np.array([0.0] * len(x_coord))
z_coord = np.array([90.0] * len(x_coord))

## backward pass: done

In [7]:
# cost from soft constraints (violating bounds on yaw angle)
u_viol_lower = F.relu(umin - u)
u_viol_upper = F.relu(u - umax)
u_viol_cost = u_penalty * torch.sum(u_viol_lower.pow(2) + u_viol_upper.pow(2))
u_viol_cost

# total cost
total_cost = u_viol_cost + power_cost
total_cost

# TO DO: sum over data in batch
opt_loss = total_cost

In [8]:
# backward pass
opt.zero_grad()
opt_loss.backward()
opt.step()

In [35]:
# model.input_layer.weight, model.hidden_layer.weight, model.output_layer.weight, \
# model.input_layer.bias, model.hidden_layer.bias, model.output_layer.bias